In [2]:
import pandas as pd
import numpy as np
import random
from random import seed


## Part 1: Import Balance Data 

In [143]:
businessbalance = pd.read_csv(r'C:\Users\johna\OneDrive\Documents\RBSI\AccountBalances.csv')

In [147]:
businessbalance.Date = pd.to_datetime(businessbalance.Date, format='DD/MM/YYYY')

In [149]:
businessbalance

,Date,Account Number,Balance,AccountType_actual,AccountType
0,2014-01-01,1111111,1.112880e+06,Limited Company,Personal
1,2014-02-01,1111111,8.275274e+05,Limited Company,Personal
2,2014-03-01,1111111,4.296631e+05,Limited Company,Personal
3,2014-04-01,1111111,2.968793e+05,Limited Company,Personal
4,2014-05-01,1111111,1.632440e+05,Limited Company,Personal
...,...,...,...,...,...
1303,2014-11-20,4444444,7.098337e+04,Limited Company,Limited Company
1304,2014-11-21,4444444,1.439251e+05,Limited Company,Limited Company
1305,2014-11-22,4444444,1.316000e+05,Limited Company,Limited Company
1306,2014-11-23,4444444,6.155685e+04,Limited Company,Limited Company


## Part 2: Generate Synthetic Balance Data for Poc

In [6]:
import sdv

In [7]:
from sdv.timeseries import PAR 

In [150]:
context_columns = ['AccountType_actual', 'AccountType']
sequence_index = 'Date'
entity_columns = ['Account Number']
#field_names = ['Balance']
#data_columns = ['Balance']

model = PAR(
    entity_columns=entity_columns,
    context_columns=context_columns,
    sequence_index=sequence_index,
    cuda=True,
    verbose=True
   
)

model.fit(businessbalance)



C:\Users\johna\anaconda3\lib\site-packages\scipy\stats\_continuous_distns.py:639: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
C:\Users\johna\anaconda3\lib\site-packages\scipy\stats\_continuous_distns.py:5320: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
C:\Users\johna\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:2606: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
C:\Users\johna\anaconda3\lib\site-packages\scipy\stats\_continuous_distns.py:5311: RuntimeWarning: divide by zero encountered in power
  return cd2*x**(c-1)
C:\Users\johna\anaconda3\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


PARModel(epochs=128, sample_size=1, cuda='cpu', verbose=True) instance created


Epoch 128 | Loss 11.800899505615234: 100%|███████████████████████████████████████████| 128/128 [00:18<00:00,  7.07it/s]


In [153]:
generatedbalance = model.sample(100)


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [05:32<00:00,  3.32s/it]


In [156]:
generatedbalance.to_csv(r'C:\Users\johna\OneDrive\Documents\RBSI\GeneratedAccountBalances.csv')

## Part 3: Generate time series features

In [154]:
from tsfresh import extract_features


In [155]:
fc_parameters = {
    "variation_coefficient": None,
    "autocorrelation":[{"lag":30}],
    "count_above_mean":None,
    "count_below_mean":None,
    "longest_strike_below_mean":None,
    "longest_strike_above_mean":None,
   # "percentage_of_reoccurring_values_to_all_datapoints":None,
    "skewness":None,
    "kurtosis":None
    
}
TimeSeriesFeatures = extract_features(timeseries_container=generatedbalance, column_id="Account Number", column_sort="Date", column_value="Balance", default_fc_parameters=fc_parameters)

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 25/25 [00:05<00:00,  4.50it/s]


In [157]:
TimeSeriesFeatures

,Balance__variation_coefficient,Balance__autocorrelation__lag_30,Balance__count_above_mean,Balance__count_below_mean,Balance__longest_strike_below_mean,Balance__longest_strike_above_mean,Balance__skewness,Balance__kurtosis
0,-3.216077,-0.093801,171.0,156.0,6.0,9.0,-0.110677,-0.075851
1,-2.485133,-0.038949,157.0,170.0,9.0,6.0,-0.154101,0.215769
2,4.824153,-0.050650,167.0,160.0,8.0,9.0,0.013867,-0.562282
3,-0.919044,-0.038922,165.0,162.0,7.0,11.0,0.057942,0.849858
4,-8.122328,0.085648,163.0,164.0,6.0,8.0,-0.203784,0.651263
...,...,...,...,...,...,...,...,...
95,-2.547156,-0.013825,157.0,170.0,8.0,9.0,0.085463,0.157863
96,-1.088233,0.039340,177.0,150.0,8.0,9.0,-0.067212,-0.090371
97,15.651903,-0.102471,171.0,156.0,12.0,9.0,0.126312,-0.077237
98,-2.431881,0.068565,173.0,154.0,8.0,8.0,-0.339051,0.237168


In [158]:
FeaturesAccounts = TimeSeriesFeatures.merge(generatedbalance[['Account Number','AccountType', 'AccountType_actual']], left_index=True, right_on='Account Number', how='inner')

## Part 4: Create K-means clusters

In [159]:
import sklearn 
from sklearn.cluster import KMeans

x = list((TimeSeriesFeatures.columns))
clusters = KMeans(n_clusters = 10).fit_predict(X=FeaturesAccounts[x])


In [160]:
clustersdf = pd.DataFrame(clusters, columns=['cluster'])
clustersdf

,cluster
0,1
1,1
2,1
3,1
4,1
...,...
32695,5
32696,5
32697,5
32698,5


In [161]:
clusteredTimeSeries = clustersdf.merge(generatedbalance[['AccountType', 'Account Number', 'AccountType_actual']], left_index=True, right_index=True, how='left').drop_duplicates()

In [162]:
clusteredTimeSeries

,cluster,AccountType,Account Number,AccountType_actual
0,1,Personal,0,Limited Company
327,0,Sole Trader,1,Sole Trader
654,8,Limited Company,2,Sole Trader
981,5,Sole Trader,3,Limited Company
1308,5,Personal,4,Sole Trader
...,...,...,...,...
31065,0,Limited Company,95,Limited Company
31392,6,Sole Trader,96,Limited Company
31719,8,Limited Company,97,Personal
32046,1,Sole Trader,98,Sole Trader


## Part 5: Analyse the clusters

In [163]:
analysis = clusteredTimeSeries.groupby(['cluster', 'AccountType']).size().unstack(fill_value=0)
analysis.to_csv(r'C:\Users\johna\OneDrive\Documents\RBSI\AnalysisGeneratedAccountBalances.csv')

In [164]:
analysis

AccountType,Limited Company,Personal,Sole Trader
cluster,,,
0,3,0,8
1,0,7,11
2,2,0,0
3,0,1,0
4,3,8,3
5,6,1,14
6,1,2,13
7,0,1,0
8,9,0,0


In [175]:
# Ouput decision cirteria

Investigate = clusteredTimeSeries.loc[(clusteredTimeSeries.cluster ==1)|(clusteredTimeSeries.cluster == 6)|(clusteredTimeSeries.cluster == 5)]
Investigate = Investigate.loc[Investigate.AccountType == "Personal"]

In [176]:
Investigate.to_csv(r'C:\Users\johna\OneDrive\Documents\RBSI\Investigate.csv')

In [172]:
clusteredTimeSeries.to_csv(r'C:\Users\johna\OneDrive\Documents\RBSI\FullClusterAnalysis.csv')